In [1]:
from datasets import load_dataset

# Load the polite dataset
dataset = load_dataset("jdustinwind/Polite")


f:\VS Code\Passive or Agressive Email rewriterr\tone_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Repo card metadata block was not found. Setting CardData to empty.


In [2]:
from datasets import DatasetDict
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token  # Set padding token

# Format each pair into "Rewrite politely: <src> <|sep|> <tgt>"
def format_example(example):
    return {
        "text": f"Rewrite politely: {example['src']} <|sep|> {example['tgt']}"
    }

# Apply formatting to all examples
dataset = dataset.map(format_example)

# Tokenize the inputs
def tokenize(example):
    return tokenizer(
        example["text"],
        truncation=True,
        padding="max_length",
        max_length=128
    )

tokenized_dataset = dataset.map(tokenize, batched=True)
tokenized_dataset.set_format(type="torch", columns=["input_ids", "attention_mask"])


f:\VS Code\Passive or Agressive Email rewriterr\tone_env\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\ALBIN JOHN\.cache\huggingface\hub\models--gpt2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Map: 100%|██████████| 100000/100000 [00:04<00:00, 21299.96 examples/s]


In [5]:
from transformers import GPT2LMHeadModel, Trainer, TrainingArguments

model = GPT2LMHeadModel.from_pretrained("gpt2")
model.resize_token_embeddings(len(tokenizer))  # Resize due to pad token

training_args = TrainingArguments(
    output_dir="./gpt2-polite",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    logging_steps=50,
    save_steps=500,
    warmup_steps=100,
    weight_decay=0.01,
    logging_dir="./logs",
    save_total_limit=2,
    fp16=True,  # Use if you have a GPU with float16
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
)


KeyError: 'test'

In [ ]:
trainer.train()
trainer.evaluate()


In [ ]:
trainer.save_model("fine-tuned-gpt2-polite")
tokenizer.save_pretrained("fine-tuned-gpt2-polite")


In [ ]:
from transformers import pipeline

generator = pipeline("text-generation", model="fine-tuned-gpt2-polite", tokenizer=tokenizer)

prompt = "Rewrite politely: Why are you always late to work? <|sep|>"
output = generator(prompt, max_length=100, do_sample=True, top_k=50, temperature=0.9)
print(output[0]['generated_text'])


In [4]:
pip install -U transformers


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
import transformers
print(transformers.__version__)

4.51.3
